In [9]:
from nbmultitask import ProcessWithLogAndControls
from IPython.display import clear_output
import time

In [10]:


from time import sleep

def fn(x):
    while True:
        print(x.value)
        x.value = x.value**1.01
        sleep(0.1)

from multiprocessing import Value
x = Value('f',2)
task = ProcessWithLogAndControls(target=fn, args=(x,), name="exponential growth")

In [5]:
task.control_panel()

Button(button_style='success', description='start', style=ButtonStyle())

Button(button_style='primary', description='watch', style=ButtonStyle())

Button(description='clear log', style=ButtonStyle())

Output()

In [11]:
task.start()
time.sleep(2)
print(x.value)
time.sleep(2)
print(x.value)
task.stop()

2.349562168121338
2.8357927799224854


In [ ]:
def test():
    import sounddevice as sd
    tone1 = get_pure_tone(440, 0.05, 44100) * 0.5

    sd.default.samplerate = 44100
    stream = sd.OutputStream(samplerate=44100, channels=2, dtype='float32')
    stream.start()

    for i in range(5):
        stream.write(np.column_stack((tone1, tone1)))
        time.sleep(0.2)
    stream.stop()

In [ ]:
selector = Value('f', 0)
is_running = Value(c_bool, True)
#sc = ProcessWithLogAndControls(target=sound_controller, args=(selector, is_running), name="sounds")
sc = ProcessWithLogAndControls(target=test, args=(), name="sounds")
sc.control_panel()

In [ ]:
def get_pure_tone(freq, duration, sample_rate=44100):
    x = np.linspace(0, duration * freq * 2*np.pi, int(duration*sample_rate), dtype=np.float32)
    return np.sin(x)

def sound_controller(selector, is_running, latency=0.25):
    """
    selector        mp.Value object to set the sound to be played
    is_running      mp.Value object to stop the loop
    """
    print('Loading SC..')
    
    tone1 = get_pure_tone(440, 0.05, 44100) * 0.5
    tone2 = get_pure_tone(880, 0.05, 44100) * 0.5
    silence = np.zeros(len(tone1), dtype='float32')    

    sounds = {
        0: np.column_stack((silence, silence)),
        1: np.column_stack((tone1, tone1)),
        2: np.column_stack((tone2, tone2))
    }

    stream = sd.OutputStream(samplerate=44100, channels=2, dtype='float32')
    stream.start()
    
    print('Creating sounds..')
        
    next_beat = time.time() + latency
    beat_times = collections.deque(maxlen=10000)

    
    while is_running.value:
        print('Running loop..')
        
        t0 = time.time()
        if t0 < next_beat:
            continue

        beat_times.append(t0)
        #sd.play(self.tone)  # has underrun errors, maybe try blocking?
        stream.write(sounds[int(selector.value)])
        next_beat += latency
        
    stream.stop()
    print('Sound controller stopped')